In [1]:
# DOWNLOAD DAS PREVISÕES METEREOLÓGICAS HORÁRIAS PARA OS DIAS DE INCÊNDIO DE 2016-2025

In [1]:
import pandas as pd
import geopandas as gpd
import os
import cdsapi
import warnings
import fiona
from shapely.geometry import shape
from datetime import datetime

In [2]:
# Carregar o ficheiro com as progressões de fogo, feature a feature e guardar apenas as válidas

shapefile = r"../../Data\Raw\PT-FireProg_v2.1\PT-FireProg_v2.1_L2.shp"

features = []

with fiona.open(shapefile, "r") as src:
    crs = src.crs
    schema = src.schema
    
    for feat in src:
        try:
            geom = shape(feat["geometry"])
            features.append({"geometry": geom, **feat["properties"]})
        except Exception as e:
            print(f"⚠️ Erro na feature {feat['id']}: {e}")

gdf = gpd.GeoDataFrame(features, crs=crs).to_crs(epsg=32629)

gdf.loc[gdf['id'] == 2717, 'edate'] = pd.to_datetime('2016-08-16 12:00:00')

os.makedirs(os.path.dirname(r"../../Data/Interim/PT-FireSprd_v2.1/L2_FireBehavior"), exist_ok=True)
gdf.to_file(r"../../Data/Interim/PT-FireSprd_v2.1/L2_FireBehavior/PT-FireSprd_v2.1_L2_valid.shp", driver="ESRI Shapefile")

print(gdf)

⚠️ Erro na feature 585: A linearring requires at least 4 coordinates.
⚠️ Erro na feature 687: A linearring requires at least 4 coordinates.
⚠️ Erro na feature 713: A linearring requires at least 4 coordinates.
⚠️ Erro na feature 743: A linearring requires at least 4 coordinates.
⚠️ Erro na feature 744: A linearring requires at least 4 coordinates.
⚠️ Erro na feature 746: A linearring requires at least 4 coordinates.
⚠️ Erro na feature 775: A linearring requires at least 4 coordinates.
⚠️ Erro na feature 853: A linearring requires at least 4 coordinates.
⚠️ Erro na feature 885: A linearring requires at least 4 coordinates.
⚠️ Erro na feature 1016: A linearring requires at least 4 coordinates.
⚠️ Erro na feature 1017: A linearring requires at least 4 coordinates.
⚠️ Erro na feature 2522: A linearring requires at least 4 coordinates.


c:\Users\dias3\miniconda3\envs\geoenv\Lib\site-packages\pyogrio\core.py:35: RuntimeWarning: Could not detect GDAL data files.  Set GDAL_DATA environment variable to the correct path.
  _init_gdal_data()
C:\Users\dias3\AppData\Local\Temp\ipykernel_7876\547593656.py:23: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(r"../../Data/Interim/PT-FireSprd_v2.1/L2_FireBehavior/PT-FireSprd_v2.1_L2_valid.shp", driver="ESRI Shapefile")
c:\Users\dias3\miniconda3\envs\geoenv\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'burn_period' to 'burn_perio'
  ogr_write(
c:\Users\dias3\miniconda3\envs\geoenv\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'growth_rate' to 'growth_rat'
  ogr_write(


                                               geometry  fid  \
0     POLYGON ((618354.847 4479040.089, 618316.236 4...    1   
1     POLYGON ((621134.68 4476676.175, 621141.065 44...    1   
2     POLYGON ((624077.929 4476623.968, 624077.929 4...    1   
3     POLYGON ((621779.415 4477648.871, 621783.225 4...    1   
4     POLYGON ((623416.198 4476954.494, 623397.368 4...    1   
...                                                 ...  ...   
3350  MULTIPOLYGON (((649699.8 4465067.381, 649688.2...  164   
3351  POLYGON ((650159.621 4458751.465, 650161.13 44...  164   
3352  POLYGON ((650634.349 4459297.623, 650539.773 4...  164   
3353  POLYGON ((652563.098 4463965.452, 652572.236 4...  164   
3354  MULTIPOLYGON (((649825.667 4456663.516, 649851...  164   

                 fname  year    id type             sdate             edate  \
0     Gouveia_10082015  2015     1    z  2015-08-10 14:30                na   
1     Gouveia_10082015  2015     2    z  2015-08-11 12:30                

In [ ]:
# Ler shapefile
shp = gpd.read_file(r"../../Data/Interim/PT-FireSprd_v2.1/L2_FireBehavior/PT-FireSprd_v2.1_L2_valid.shp")

# Converter datas
shp["sdate"] = pd.to_datetime(shp["sdate"], errors='coerce')
shp["edate"] = pd.to_datetime(shp["edate"], errors='coerce')

# Criar lista de horas
horas_list = []

for idx, row in shp.iterrows():
    sdate = row['sdate']
    edate = row['edate']
    
    if pd.notna(sdate) and pd.notna(edate):
        # Ambos presentes: todas as horas cheias no intervalo
        horas_list.append(pd.date_range(start=sdate.floor('h'), end=edate.ceil('h'), freq='h'))

# Unir todas as horas e remover duplicados
time_list = sorted({hora for sublist in horas_list for hora in sublist})

print(f"Total de timestamps únicos: {len(time_list)}")
print(time_list)

pd.DataFrame(time_list, columns=["datetime"]).to_csv(r"..\..\Data\Interim\Meteorological_data\time_list.csv", index=False)

# Criar DataFrame com os dias únicos
date_list = pd.DataFrame({"datetime": time_list})
date_list["day"] = date_list["datetime"].dt.floor("D")
date_list = sorted(date_list["day"].unique())

print(f"Número de dias únicos: {len(date_list)}")
print(date_list)

C:\Users\dias3\AppData\Local\Temp\ipykernel_6616\2460082361.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  shp["edate"] = pd.to_datetime(shp["edate"], errors='coerce')


Total de timestamps únicos: 2486
[Timestamp('2015-08-03 14:00:00'), Timestamp('2015-08-03 15:00:00'), Timestamp('2015-08-03 16:00:00'), Timestamp('2015-08-03 17:00:00'), Timestamp('2015-08-03 18:00:00'), Timestamp('2015-08-03 19:00:00'), Timestamp('2015-08-03 20:00:00'), Timestamp('2015-08-03 21:00:00'), Timestamp('2015-08-03 22:00:00'), Timestamp('2015-08-03 23:00:00'), Timestamp('2015-08-04 00:00:00'), Timestamp('2015-08-04 01:00:00'), Timestamp('2015-08-04 02:00:00'), Timestamp('2015-08-04 03:00:00'), Timestamp('2015-08-08 12:00:00'), Timestamp('2015-08-08 13:00:00'), Timestamp('2015-08-08 14:00:00'), Timestamp('2015-08-08 15:00:00'), Timestamp('2015-08-08 16:00:00'), Timestamp('2015-08-08 17:00:00'), Timestamp('2015-08-08 18:00:00'), Timestamp('2015-08-08 19:00:00'), Timestamp('2015-08-08 20:00:00'), Timestamp('2015-08-08 21:00:00'), Timestamp('2015-08-08 22:00:00'), Timestamp('2015-08-08 23:00:00'), Timestamp('2015-08-09 00:00:00'), Timestamp('2015-08-09 13:00:00'), Timestamp('201

In [5]:
# Agrupar os dias de incêndio por ano/mês

df = pd.DataFrame({
    "year": [d.year for d in date_list],
    "month": [d.month for d in date_list],
    "day": [d.day for d in date_list],
})

df_grouped = (
    df.groupby(["year", "month"])["day"]
    .apply(lambda x: sorted(x.unique()))
    .reset_index()
    .rename(columns={"day": "days"})
)

df_grouped["year"] = df_grouped["year"].astype(int)
df_grouped["month"] = df_grouped["month"].astype(int)

df_grouped["days"] = df_grouped["days"].apply(lambda lst: [int(d) for d in lst])

print("Dias de Incêndio:")
print(df_grouped)

Dias de Incêndio:
    year  month                                               days
0   2015      8                           [3, 4, 8, 9, 10, 11, 12]
1   2016      8                      [7, 8, 9, 10, 11, 12, 13, 14]
2   2016      9                                       [5, 6, 7, 8]
3   2017      6                           [17, 18, 19, 20, 21, 22]
4   2017      7               [16, 17, 18, 23, 24, 25, 26, 27, 28]
5   2017      8  [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 23...
6   2017      9                                             [8, 9]
7   2017     10                        [5, 6, 7, 8, 9, 15, 16, 17]
8   2018      8                                 [3, 4, 5, 6, 7, 8]
9   2019      6                                               [10]
10  2019      7                               [20, 21, 22, 23, 24]
11  2019      8                                         [1, 3, 29]
12  2019      9                                  [4, 5, 6, 13, 14]
13  2020      6                             

In [6]:
# Conta o número total de dias de incêndio (somando todos os meses/anos)
total_dias = df_grouped["days"].apply(len).sum()

print(f"Total de dias de incêndio: {total_dias}")

Total de dias de incêndio: 202


In [7]:
import pandas as pd
from pathlib import Path

# Pasta de output
output_folder = Path(r"../../Data/Raw/Meteorological_data/ERA5_Land_downloads")

# Analisar arquivos existentes
existing_sl = set()
existing_pl = set()
existing_land = set()
existing_fb = set()
existing_fwi = set()

for file_path in output_folder.glob("ERA5_SL_*.nc"):
    year = int(file_path.name[8:12])
    month = int(file_path.name[12:14])
    existing_sl.add((year, month))

for file_path in output_folder.glob("ERA5_PL_*.nc"):
    year = int(file_path.name[8:12])
    month = int(file_path.name[12:14])
    existing_pl.add((year, month))

for file_path in output_folder.glob("ERA5_Land_*.nc"):
    year = int(file_path.name[10:14])
    month = int(file_path.name[14:16])
    existing_land.add((year, month))

for file_path in output_folder.glob("ERA5_fb_*.nc"):
    year = int(file_path.name[8:12])
    month = int(file_path.name[12:14])
    existing_fb.add((year, month))

for file_path in output_folder.glob("ERA5_FWI_*.nc"):
    year = int(file_path.name[9:13])
    month = int(file_path.name[13:15])
    existing_fwi.add((year, month))

# Filtrar dados que faltam
df_grouped_sl_missing = df_grouped[~df_grouped[['year', 'month']].apply(tuple, axis=1).isin(existing_sl)]
df_grouped_pl_missing = df_grouped[~df_grouped[['year', 'month']].apply(tuple, axis=1).isin(existing_pl)]
df_grouped_land_missing = df_grouped[~df_grouped[['year', 'month']].apply(tuple, axis=1).isin(existing_land)]
df_grouped_fb_missing = df_grouped[~df_grouped[['year', 'month']].apply(tuple, axis=1).isin(existing_fb)]
df_grouped_fwi_missing = df_grouped[~df_grouped[['year', 'month']].apply(tuple, axis=1).isin(existing_fwi)]


print("Meses faltantes para ERA5_SL:")
print(df_grouped_sl_missing)

print("\nMeses faltantes para ERA5_PL:")
print(df_grouped_pl_missing)

print("\nMeses faltantes para ERA5_Land:")
print(df_grouped_land_missing)

print("\nMeses faltantes para ds_Fb:")
print(df_grouped_fb_missing)

print("\nMeses faltantes para ds_FWI:")
print(df_grouped_fwi_missing)

Meses faltantes para ERA5_SL:
Empty DataFrame
Columns: [year, month, days]
Index: []

Meses faltantes para ERA5_PL:
Empty DataFrame
Columns: [year, month, days]
Index: []

Meses faltantes para ERA5_Land:
Empty DataFrame
Columns: [year, month, days]
Index: []

Meses faltantes para ds_Fb:
Empty DataFrame
Columns: [year, month, days]
Index: []

Meses faltantes para ds_FWI:
    year  month                                               days
0   2015      8                           [3, 4, 8, 9, 10, 11, 12]
1   2016      8                      [7, 8, 9, 10, 11, 12, 13, 14]
2   2016      9                                       [5, 6, 7, 8]
3   2017      6                           [17, 18, 19, 20, 21, 22]
4   2017      7               [16, 17, 18, 23, 24, 25, 26, 27, 28]
5   2017      8  [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 23...
6   2017      9                                             [8, 9]
7   2017     10                        [5, 6, 7, 8, 9, 15, 16, 17]
8   2018      8         

In [8]:
max_len = 10

def split_dataframe(df):
    new_rows = []
    for _, row in df.iterrows():
        days = row["days"]
        if len(days) > max_len:
            for i in range(0, len(days), max_len):
                new_rows.append({
                    "year": row["year"],
                    "month": row["month"],
                    "days": days[i:i+max_len]
                })
        else:
            new_rows.append(row.to_dict())
    return pd.DataFrame(new_rows)

# Aplicar split a todos os DataFrames
df_grouped_sl_missing_split = split_dataframe(df_grouped_sl_missing)
df_grouped_pl_missing_split = split_dataframe(df_grouped_pl_missing)
df_grouped_land_missing_split = split_dataframe(df_grouped_land_missing)
df_grouped_fb_missing_split = split_dataframe(df_grouped_fb_missing)

print("Meses faltantes para ERA5_SL (split):")
print(df_grouped_sl_missing_split)

print("\nMeses faltantes para ERA5_PL (split):")
print(df_grouped_pl_missing_split)

print("\nMeses faltantes para ERA5_Land (split):")
print(df_grouped_land_missing_split)

print("\nMeses faltantes para ds_Fb (split):")
print(df_grouped_fb_missing_split)

Meses faltantes para ERA5_SL (split):
Empty DataFrame
Columns: []
Index: []

Meses faltantes para ERA5_PL (split):
Empty DataFrame
Columns: []
Index: []

Meses faltantes para ERA5_Land (split):
Empty DataFrame
Columns: []
Index: []

Meses faltantes para ds_Fb (split):
Empty DataFrame
Columns: []
Index: []


In [9]:
client = cdsapi.Client()

In [10]:
warnings.filterwarnings("default")

In [11]:
os.makedirs(r"../../Data/Raw/Meteorological_data/ERA5_Land_downloads", exist_ok=True)
os.makedirs(r"../../Data/Interim/Meteorological_data/ERA5_NetCDF", exist_ok=True)
os.makedirs(r"../../Data/Interim/GIS_data", exist_ok=True)

In [12]:
# Faz os requests ao CDS da info para todas as horas dos dias anteriormente definidos, para o pais todo
output_folder = Path(r"../../Data/Raw/Meteorological_data/ERA5_Land_downloads")
time = list(range(24))

if df_grouped_sl_missing_split.empty:
    print("Ficheiros já existem, skipping...")

dataset = "reanalysis-era5-single-levels"

for idx, row in df_grouped_sl_missing_split.iterrows():
    year = str(row['year'])
    month = f"{row['month']:02d}"
    days = [d for d in row['days']]

    request = {
        "product_type": ["reanalysis"],
        "variable": [
            "100m_u_component_of_wind",
            "100m_v_component_of_wind",
            "cloud_base_height",
            "high_cloud_cover",
            "low_cloud_cover",
            "medium_cloud_cover",
            "total_cloud_cover",
            "boundary_layer_height",
            "convective_available_potential_energy",
            "convective_inhibition",
            "volumetric_soil_water_layer_1",
            "volumetric_soil_water_layer_2",
            "volumetric_soil_water_layer_3",
            "volumetric_soil_water_layer_4"
        ],
        "year": year,
        "month": month,
        "day": days,
        "time": time,
        "data_format": "netcdf",
        "download_format": "unarchived",
        "area": [43, -10, 37, -6]
    }

    timestamp = datetime.now().strftime("%H%M%S")
    filename = f"ERA5_SL_{year}{month}_{timestamp}.nc"
    target = output_folder / filename

    print("Requesting:", filename)
    client.retrieve(dataset, request, str(target))


Ficheiros já existem, skipping...


In [13]:
# Faz os requests ao CDS da info para todas as horas dos dias anteriormente definidos, para o pais todo
output_folder = Path(r"../../Data/Raw/Meteorological_data/ERA5_Land_downloads")
time = list(range(24))

if df_grouped_pl_missing_split.empty:
    print("Ficheiros já existem, skipping...")

dataset = "reanalysis-era5-pressure-levels"

for idx, row in df_grouped_pl_missing_split.iterrows():
    year = str(row['year'])
    month = f"{row['month']:02d}"
    days = [d for d in row['days']]

    request = {
        "product_type": ["reanalysis"],
        "variable": [
            "geopotential",
            "relative_humidity",
            "temperature",
            "u_component_of_wind",
            "v_component_of_wind",
            "vertical_velocity"
        ],
        "pressure_level": ["950", "850", "700", "500", "300"],
        "year": year,
        "month": month,
        "day": days,
        "time": time,
        "data_format": "netcdf",
        "download_format": "unarchived",
        "area": [43, -10, 37, -6]
    }

    timestamp = datetime.now().strftime("%H%M%S")
    filename = f"ERA5_PL_{year}{month}_{timestamp}.nc"
    target = output_folder / filename

    print("Requesting:", filename)
    client.retrieve(dataset, request, str(target))

Ficheiros já existem, skipping...


In [14]:
# Faz os requests ao CDS da info para todas as horas dos dias anteriormente definidos, para o pais todo
output_folder = Path(r"../../Data/Raw/Meteorological_data/ERA5_Land_downloads")
time = list(range(24))

if df_grouped_land_missing_split.empty:
    print("Ficheiros já existem, skipping...")

dataset = "reanalysis-era5-land"

for idx, row in df_grouped_land_missing_split.iterrows():
    year = str(row['year'])
    month = f"{row['month']:02d}"
    days = [d for d in row['days']]

    request = {
        "product_type": ["reanalysis"],
        "variable": [
            "2m_dewpoint_temperature",
            "2m_temperature",
            "10m_u_component_of_wind",
            "10m_v_component_of_wind",
            "surface_pressure"
        ],
        "year": year,
        "month": month,
        "day": days,
        "time": time,
        "data_format": "netcdf",
        "download_format": "unarchived",
        "area": [43, -10, 37, -6]
    }

    timestamp = datetime.now().strftime("%H%M%S")
    filename = f"ERA5_Land_{year}{month}_{timestamp}.nc"
    target = output_folder / filename

    print("Requesting:", filename)
    client.retrieve(dataset, request, str(target))


Ficheiros já existem, skipping...


In [15]:
# Faz os requests ao CDS da info para todas as horas dos dias anteriormente definidos, para o pais todo
output_folder = Path(r"../../Data/Raw/Meteorological_data/ERA5_Land_downloads")
time = list(range(24))

if df_grouped_fb_missing_split.empty:
    print("Ficheiros já existem, skipping..."),

dataset = "reanalysis-era5-single-levels"

for idx, row in df_grouped_fb_missing_split.iterrows():
    year = str(row['year'])
    month = f"{row['month']:02d}"
    days = [d for d in row['days']]

    request = {
        "product_type": ["reanalysis"],
        "variable": [
            "2m_dewpoint_temperature",
            "2m_temperature",
            "10m_u_component_of_wind",
            "10m_v_component_of_wind",
            "surface_pressure"
        ],
        "year": year,
        "month": month,
        "day": days,
        "time": time,
        "data_format": "netcdf",
        "download_format": "unarchived",
        "area": [43, -10, 37, -6]
    }

    timestamp = datetime.now().strftime("%H%M%S")
    filename = f"ERA5_Fb_{year}{month}_{timestamp}.nc"
    target = output_folder / filename

    print("Requesting:", filename)
    client.retrieve(dataset, request, str(target))


Ficheiros já existem, skipping...


In [16]:
# Faz os requests ao CDS da info para todas as horas dos dias anteriormente definidos, para o pais todo
output_folder = Path(r"../../Data/Raw/Meteorological_data/ERA5_Land_downloads")
time = list(range(24))

# Verificar se já existe algum ficheiro com "GP"
gp_files_exist = any("GP" in file.name for file in output_folder.glob("*.nc"))

if not gp_files_exist:
    dataset = "reanalysis-era5-land"
    
    request = {
        "product_type": ["reanalysis"],
        "variable": [
            "geopotential"
        ],
        "year": "2013",
        "month": "08",
        "day": ["09"],
        "time": ["12:00"],
        "data_format": "netcdf",
        "download_format": "unarchived",
        "area": [43, -10, 37, -6]
    }

    timestamp = datetime.now().strftime("%H%M%S")
    filename = f"ERA5_GP_201308_{timestamp}.nc"
    target = output_folder / filename

    print("Requesting:", filename)
    client.retrieve(dataset, request, str(target))
else:
    print("Ficheiro GP já existe, skipping...")

Ficheiro GP já existe, skipping...


In [17]:
# Faz os requests ao CDS da info para todas as horas dos dias anteriormente definidos, para o pais todo
output_folder = Path(r"../../Data/Raw/Meteorological_data/ERA5_Land_downloads")
time = list(range(24))

if df_grouped_fwi_missing.empty:
    print("Ficheiros já existem, skipping...")

dataset = "cems-fire-historical-v1"

for idx, row in df_grouped_fwi_missing.iterrows():
    year = str(row['year'])
    month = f"{row['month']:02d}"
    days = [f"{d:02d}" for d in row['days']]

    request = {
        "product_type": "reanalysis",
        "variable": [
            "drought_code",
            "fine_fuel_moisture_code",
            "fire_weather_index"
        ],
        "dataset_type": "consolidated_dataset",
        "system_version": "4_1",
        "year": year,
        "month": month,
        "day": days,
        "grid": "original_grid",
        "data_format": "netcdf"
    }

    timestamp = datetime.now().strftime("%H%M%S")
    filename = f"ERA5_FWI_{year}{month}_{timestamp}.nc"
    target = output_folder / filename

    print("Requesting:", filename)
    client.retrieve(dataset, request).download(str(target))

Requesting: ERA5_FWI_201508_175956.nc


2025-11-17 17:59:56,615 INFO Request ID is 877fdc68-1c09-49ce-9318-d1b48b636a38
2025-11-17 17:59:56,705 INFO status has been updated to accepted


2025-11-17 18:00:18,250 INFO status has been updated to running
2025-11-17 18:00:46,867 INFO status has been updated to successful


49ce0f3ba823622ebe197d056f7d4e48.nc:   0%|          | 0.00/15.1M [00:00<?, ?B/s]

2025-11-17 18:00:56,466 INFO Request ID is ef676ea4-4b67-4328-b9ac-f446caa7954a


...

2025-11-17 18:11:46,260 INFO Request ID is 2ff71a40-1401-487f-8c7b-3d88a1b94d1a
2025-11-17 18:11:46,350 INFO status has been updated to accepted
2025-11-17 18:12:07,715 INFO status has been updated to successful


7060ca2c77465cfb4b8dbd321f005347.nc:   0%|          | 0.00/21.8M [00:00<?, ?B/s]

Requesting: ERA5_FWI_202508_181216.nc


2025-11-17 18:12:16,909 INFO Request ID is 6e098aaa-1e14-4e6c-8dde-ee73e79574c2
2025-11-17 18:12:17,004 INFO status has been updated to accepted
2025-11-17 18:12:38,235 INFO status has been updated to running
2025-11-17 18:12:49,694 INFO status has been updated to successful


61bd3fc2fa56d61ede83d2b187767df4.nc:   0%|          | 0.00/22.8M [00:00<?, ?B/s]